In [1]:
from datetime import datetime
from meteostat import Point, Daily, Hourly
import pandas as pd
from meteostat import Stations

In [2]:
current_date = datetime.now()
start = datetime(current_date.year -1, current_date.month, current_date.day)
end = datetime(current_date.year, current_date.month, current_date.day, current_date.hour-1, current_date.minute, current_date.second)

print(start)
print(end)

2024-08-27 00:00:00
2025-08-27 17:46:41


In [3]:
from retrieval import get_coordinates

In [4]:
lat, lng = get_coordinates("Calw")
lat

48.7127

In [5]:
stations = Stations()
stations = stations.nearby(lat, lng)
station = stations.fetch(1)

station

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
7M2XN,Calw-hirsau,DE,BW,<NA>,<NA>,48.745,8.7301,329.0,Europe/Berlin,2018-05-23,2025-08-14,NaT,NaT,NaT,NaT,3704.963251


In [6]:
id = station.index.to_numpy()[0]
id

'7M2XN'

In [7]:
data = Hourly(id, start, end)
data = data.fetch()

df = pd.DataFrame(data)
df.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2024-08-27 00:00:00,14.8,8.3,65.0,0.0,<NA>,1.0,7.6,<NA>,1022.2,<NA>,1.0
2024-08-27 01:00:00,14.9,8.4,65.0,0.0,<NA>,21.0,6.8,<NA>,1021.9,<NA>,1.0
2024-08-27 02:00:00,16.0,8.2,60.0,0.0,<NA>,28.0,6.5,<NA>,1021.6,<NA>,1.0
2024-08-27 03:00:00,15.7,8.4,62.0,0.0,<NA>,60.0,7.2,<NA>,1021.5,<NA>,1.0
2024-08-27 04:00:00,15.8,8.8,63.0,0.0,<NA>,81.0,8.3,<NA>,1021.5,<NA>,1.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8778 entries, 2024-08-27 00:00:00 to 2025-08-27 17:00:00
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   temp    8419 non-null   Float64
 1   dwpt    8419 non-null   Float64
 2   rhum    8419 non-null   Float64
 3   prcp    8778 non-null   Float64
 4   snow    0 non-null      Float64
 5   wdir    8419 non-null   Float64
 6   wspd    8419 non-null   Float64
 7   wpgt    0 non-null      Float64
 8   pres    8419 non-null   Float64
 9   tsun    0 non-null      Float64
 10  coco    8347 non-null   Float64
dtypes: Float64(11)
memory usage: 917.2 KB


In [9]:
na_cols = [col for col in df.columns if df[col].count() < 1000]
df = df.drop(na_cols, axis = 1)

In [28]:
df.dropna(inplace=True)

In [29]:
# coco and temperature can be the dependent variables

In [30]:
coco_dict = {
    1: "Clear",
    2: "Fair",
    3: "Cloudy",
    4: "Overcast",
    5: "Fog",
    6: "Freezing Fog",
    7: "Light Rain",
    8: "Rain",
    9: "Heavy Rain",
    10: "Freezing Rain",
    11: "Heavy Freezing Rain",
    12: "Sleet",
    13: "Heavy Sleet",
    14: "Light Snowfall",
    15: "Snowfall",
    16: "Heavy Snowfall",
    17: "Rain Shower",
    18: "Heavy Rain Shower",
    19: "Sleet Shower",
    20: "Heavy Sleet Shower",
    21: "Snow Shower",
    22: "Heavy Snow Shower",
    23: "Lightning",
    24: "Hail",
    25: "Thunderstorm",
    26: "Heavy Thunderstorm",
    27: "Storm"
}


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
y = df["coco"]
X = df.drop("coco", axis = 1)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

In [34]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6677, 7), (1670, 7), (6677,), (1670,))

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [96]:
from sklearn.neural_network import MLPClassifier

In [90]:
forest = RandomForestClassifier(n_estimators=100)

In [134]:
mlpc = MLPClassifier(random_state=42, max_iter=50000)

In [135]:
mlpc.fit(X_train, y_train)

,hidden_layer_sizes,"(100,)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,50000
,shuffle,True
,random_state,42


In [136]:
forest.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [137]:
test_result = pd.DataFrame({
    "prediction" : forest.predict(X_test),
    "actual" : y_test,
    }
)

In [138]:
test_result2 = pd.DataFrame({
    "prediction" : mlpc.predict(X_test),
    "actual" : y_test,
    }
)

In [139]:
test_result["correct"] = test_result["actual"] == test_result["prediction"]

In [140]:
test_result2["correct"] = test_result2["actual"] == test_result2["prediction"]

In [141]:
correct = len(test_result[test_result["correct"] == True])
total = len(test_result)

In [142]:
correct2 = len(test_result2[test_result2["correct"] == True])
total2 = len(test_result2)

In [143]:
print(f"{correct} out of {total} predictions are correct.\n {(correct/total):.2f} % success rate ")

1089 out of 1670 predictions are correct.
 0.65 % success rate 


In [144]:
print(f"{correct2} out of {total2} predictions are correct.\n {(correct2/total2):.2f} % success rate ")

667 out of 1670 predictions are correct.
 0.40 % success rate 
